In [ ]:
#This file actually finds the eddies from the SLA files

In [1]:
#FILL IN INFORMATION HERE
from datetime import datetime
from datetime import timedelta, date

filepath="rawdata/" #path to netcdf file with SSH

tmax=30 #total number of days
savepath="eddies/" #path to save the eddies to
imagedir='eddyimages/' #path to save the images to
filepath1="rawdata2/" #path to the preprocessed (for eddy detection) AVISO data
filepath2="/home/mavida/dat1/miniconda3/eddytracking/testsonscottsdata/rawdata2/" #path to Scott's preprocessed (for eddy detection) predictions)
slice_=False
longmin=360-62 #minimum longitude
longmax=360-50 #maximum longitude
latmin=25 #minimum latitude
latmax=52 #maximum latitude
def findfilename(path,t):
    return path+str(t)+".nc"
startdate=datetime(2021,11,15)


In [2]:
#Make sure py eddy tracker is installed; see https://py-eddy-tracker.readthedocs.io/en/stable/installation.html
from matplotlib import pyplot as plt
from numpy import arange
import numpy as np

from py_eddy_tracker import data
from py_eddy_tracker.dataset.grid import RegularGridDataset
from py_eddy_tracker.eddy_feature import Contours
from datetime import timedelta, date

from py_eddy_tracker.data import get_remote_demo_sample
from py_eddy_tracker.featured_tracking.area_tracker import AreaTracker
from py_eddy_tracker.gui import GUI
from py_eddy_tracker.tracking import Correspondances

import logging
from matplotlib import pyplot as plt
from netCDF4 import Dataset

from py_eddy_tracker import start_logger
from py_eddy_tracker.data import get_remote_demo_sample
from py_eddy_tracker.featured_tracking.area_tracker import AreaTracker

# In order to hide some warning
import py_eddy_tracker.observations.observation
from py_eddy_tracker.tracking import Correspondances

py_eddy_tracker.observations.observation._display_check_warning = False

from datetime import datetime

from matplotlib import pyplot as plt
from numpy import arange
import numpy as np

from py_eddy_tracker import data
from py_eddy_tracker.dataset.grid import RegularGridDataset
from py_eddy_tracker.eddy_feature import Contours
from datetime import timedelta, date


In [3]:
#coordinate names - Fill in
xname="longitude" #name of x coordinate
yname="latitude" #name of y coordinate
tname="time"

margin=30
file_objects=[]
date_=startdate

In [ ]:
#This saves the eddy observations for the AVISO data and Scott's data so that we can track the eddies and conduct analyses

for t in range(tmax):
    for filepath,savepath in [[filepath1,"AVISO"],[filepath2,"ML"]]:    
        filename=findfilename(filepath,t)
        g =RegularGridDataset(filename,xname,yname,nan_masking=True)
        g.add_uv("adt") #calculates u and v based on sea surface height

        a, c = g.eddy_identification("adt", "u", "v", date_, 0.002, shape_error=55) #identifies the eddies
        a.write_file(path=savepath, filename=savepath+str(t)+"a.nc", zarr_flag=False) #saves the anticyclonic eddies
        c.write_file(path=savepath, filename=savepath+str(t)+"c.nc", zarr_flag=False) #saves the cyclonic eddies
    

In [ ]:
#This function finds the eddy observations for each day in the range and saves them as images which can then be turned into a video using ffmpeg
from matplotlib.animation import FuncAnimation  #creating animations and videos.
from IPython.display import Video, HTML
import matplotlib as mpl
def animation_func(i,filepath1,filepath2,savedir):
    filename1=findfilename(filepath1,i)
    filename2=findfilename(filepath2,i)
    fig, axs = plt.subplot_mosaic([['BLANK', 'BLANK', 'BLANK','BLANK'],['ax1', 'ax1','ax2','ax2'],
     ['ax1', 'ax1','ax2','ax2'],['colorbar', 'colorbar','colorbar','colorbar']],empty_sentinel="BLANK",figsize=(10,10))
    date_= startdate+timedelta(days=i)
    cmap = mpl.cm.viridis
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=axs['colorbar'], orientation='horizontal', label='ADT')
    for filename,ax,add,name in [[filename1,axs['ax1'],360,"AVISO data"],[filename2,axs['ax2'],0, "ML Interpolated Data"]]:
        print(filename,ax)
        loc=ax.get_position()
        #ax.set_position([loc.x0, loc.y0, loc.width, loc.height*0.75])
        
        
        ax.title.set_text(name)
        g =RegularGridDataset(filename,xname,yname,nan_masking=True)
        
        print(g)
        g.add_uv("adt") #calculates u and v based on sea surface height
        
        g.bessel_high_filter("adt", 700) #highlights the mesoscale
        
        a, c = g.eddy_identification("adt", "u", "v", date_, 0.002, shape_error=70) #identifies the eddies
        im = ax.pcolormesh(g.x_bounds+add, g.y_bounds, g.grid("adt").T,cmap=cmap,norm=norm)
        
        a.display(
            ax, color="r", linewidth=0.75, label="Anticyclonic ({nb_obs} eddies)", ref=-10
        )
        c.display(ax, color="b", linewidth=0.75, label="Cyclonic ({nb_obs} eddies)", ref=-10)
        ax.legend(bbox_to_anchor=(0., 1.1, 1., .102), loc='lower left',
                       mode="expand", borderaxespad=0.)

    
    plt.savefig(savedir+(str(i).zfill(2))+'.jpg')
    
for t in range(tmax):
    animation_func(t,filepath1,filepath2,imagedir)